# Dfs0

In [ ]:
import numpy as np
import pandas as pd
import mikeio

## Reading data

In [ ]:
ds = mikeio.read("data/TemporalEqTime.dfs0")
ds

In [ ]:
type(ds)

The MIKE IO `Dataset` are used by all Dfs classes (Dfs0,Dfs1,Dfs2,Dfs3, Dfsu) but Dfs0 data are best handled by a Pandas DataFrame.

In [ ]:
df = ds.to_dataframe() # convert dataset to dataframe
df

## Writing data


In [ ]:
df = pd.read_csv("data/naples_fl.csv", skiprows=1, parse_dates=True, index_col=0)
df

You will probably have the need to parse certain a specific data formats many times, then it is a good idea to create a function.

In [ ]:
def read_ncei_obs(filename):
    """Parse Meteo observations from NCEI"""
    
    sel_cols = ['temperature_avg_f','temperature_max_f','temperature_min_f', 'prec_in']
    df = (
        pd.read_csv("data/naples_fl.csv", skiprows=1, parse_dates=True, index_col=0)
           .rename(columns={'TAVG (Degrees Fahrenheit)': 'temperature_avg_f',
                            'TMAX (Degrees Fahrenheit)': 'temperature_max_f',
                            'TMIN (Degrees Fahrenheit)': 'temperature_min_f',
                            'PRCP (Inches)': 'prec_in'})
    )[sel_cols]
    
    df.index = pd.DatetimeIndex(df.index, freq='infer') # detect equidistant
    
    return df

In [ ]:
df = read_ncei_obs("data/naples_fl.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df['temperature_max_c'] = (df['temperature_max_f'] - 32)/1.8
df['prec_mm'] = df['prec_in'] * 25.4
df.head()

In [ ]:
df.loc['2021'].plot()

The simplest way to create a dfs0 file is to use the `to_dfs0` method on a Pandas dataframe.

In [ ]:
df.to_dfs0("output/naples_fl.dfs0")

Let's read it back in again...

In [ ]:
saved_ds = mikeio.read("output/naples_fl.dfs0")
saved_ds

By default, EUM types are undefined. But it can be specified.

In [ ]:
from mikeio.eum import EUMType

df['temperature_max_c'].to_dfs0("output/naples_fl_eum.dfs0", itemtype=EUMType.Temperature)

In [ ]:
mikeio.read("output/naples_fl_eum.dfs0")

## EUM

In [ ]:
from mikeio.eum import ItemInfo, EUMType, EUMUnit

EUMType.search("wind")


In [ ]:
EUMType.Wind_speed.units

## Precipitation data

In [ ]:
df = pd.read_csv("data/precipitation.csv", parse_dates=True, index_col=0)
df.head()

Using a [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) is a compact way to manipulate data similar to using a for loop.

In [ ]:
squares = [x**2 for x in range(10)]
squares

In [ ]:
from mikecore.DfsFile import DataValueType

items = [ItemInfo(name, EUMType.Precipitation_Rate, EUMUnit.mm_per_hour, data_value_type=DataValueType.MeanStepBackward) for name in df.columns]

items

In [ ]:
from string import ascii_uppercase

def create_prec_item(raw_name):
    """Create a item info with clean short name and correct EUM"""
    
    idx = int(raw_name[-1]) - 1
    
    name = (raw_name.replace("Precipitation ","")
                     .replace(" ", "_")
                     .capitalize()
                     .replace(raw_name[-1], ascii_uppercase[idx])
           )
    
    iteminfo = ItemInfo(name, EUMType.Precipitation_Rate, EUMUnit.mm_per_hour, data_value_type=DataValueType.MeanStepBackward)
    return iteminfo
    

In [ ]:
create_prec_item("Precipitation station 9")

In [ ]:
items = [create_prec_item(name) for name in df.columns]

items

In [ ]:
items[0].data_value_type

In [ ]:
df.to_dfs0("output/precipitation.dfs0", items=items)

## Selecting items

In [ ]:
ds = mikeio.read("output/precipitation.dfs0", items=[1,4]) # select item by item number (starting from zero)
ds

In [ ]:
ds = mikeio.read("output/precipitation.dfs0", items=["Station_E","Station_B"]) # or by name (in the order you like it)
ds

In [ ]:
ds["Station_E"]